In [17]:
import pandas as pd
from pathlib import Path


In [18]:
d1 = pd.read_csv("resumes_dataset1_clean.csv")
d2 = pd.read_csv("resumes_dataset2_clean.csv")

print("Dataset 1 limpio:", d1.shape)
print("Dataset 2 limpio:", d2.shape)

Dataset 1 limpio: (8076, 5)
Dataset 2 limpio: (188, 5)


In [19]:
d1["source_dataset"] = "dataset1_avishek"
d2["source_dataset"] = "dataset2_jithin"

In [20]:
# eliminar role_label_final antiguo
if "role_label_final" in d1.columns:
    d1 = d1.drop(columns=["role_label_final"])
if "role_label_final" in d2.columns:
    d2 = d2.drop(columns=["role_label_final"])

In [21]:
# unir los dos datasets limpios
full = pd.concat([d1, d2], ignore_index=True)
print("Total filas combinadas:", full.shape)

Total filas combinadas: (8264, 4)


In [22]:
# cargar mapping ya con role_label_final
mapping = pd.read_csv("role_mapping_manual.csv")
print(mapping)

                              role_raw    source_dataset  \
0               Database_Administrator  dataset1_avishek   
1                  Front_End_Developer  dataset1_avishek   
2                       Java_Developer  dataset1_avishek   
3                Network_Administrator  dataset1_avishek   
4                      Project_manager  dataset1_avishek   
5                     Python_Developer  dataset1_avishek   
6                     Security_Analyst  dataset1_avishek   
7                   Software_Developer  dataset1_avishek   
8                Systems_Administrator  dataset1_avishek   
9                        Web_Developer  dataset1_avishek   
10                   Backend Developer   dataset2_jithin   
11                      Cloud Engineer   dataset2_jithin   
12                      Data Scientist   dataset2_jithin   
13                  Frontend Developer   dataset2_jithin   
14                Full Stack Developer   dataset2_jithin   
15           Machine Learning Engineer  

In [23]:
full_mapped = full.merge(
    mapping,
    how="left",
    on=["role_raw", "source_dataset"]
)

print("Filas sin mapeo:",
      full_mapped["role_label_final"].isna().sum())

# Si esto no es 0, mostramos qué roles faltan
if full_mapped["role_label_final"].isna().sum() > 0:
    display(
        full_mapped[full_mapped["role_label_final"].isna()][["role_raw", "source_dataset"]].drop_duplicates()
    )


Filas sin mapeo: 0


In [24]:
# crear id
full_mapped["cv_id"] = range(1, len(full_mapped) + 1)

# seleccionar columnas finales
cv_final = full_mapped[[
    "cv_id",
    "cv_text",
    "role_label_final",
    "role_raw",
    "source_dataset",
    "resume_len"
]]

print(cv_final.shape)
cv_final.head()


(8264, 6)


,cv_id,cv_text,role_label_final,role_raw,source_dataset,resume_len
0,1,Python Developer Python Developer Python Devel...,python_developer,Python_Developer,dataset1_avishek,3467
1,2,R&D Engineer R&D Engineer R&D Engineer - Nokia...,python_developer,Python_Developer,dataset1_avishek,2812
2,3,Sr. Full Stack Developer Sr. Full Stack Develo...,python_developer,Python_Developer,dataset1_avishek,16606
3,4,Sr. Full Stack Python Developer Sr. Full Stack...,python_developer,Python_Developer,dataset1_avishek,9253
4,5,Sr. Python Developer Sr. Python Developer Sr. ...,python_developer,Python_Developer,dataset1_avishek,15659


In [25]:
out_path = Path("../3.processed/cv_labeled_final.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
cv_final.to_csv(out_path, index=False)

print("Guardado dataset final en:", out_path)
print("Total filas:", cv_final.shape[0])

print("\nDistribución final de roles:")
print(cv_final["role_label_final"].value_counts())


Guardado dataset final en: ..\3.processed\cv_labeled_final.csv
Total filas: 8264

Distribución final de roles:
role_label_final
python_developer          1292
java_developer            1194
web_developer              958
database_administrator     917
security_analyst           865
systems_administrator      764
project_manager            719
frontend_developer         547
network_administrator      452
software_developer         414
fullstack_developer         30
data_scientist              29
mobile_app_developer        23
cloud_engineer              23
backend_developer           19
ml_engineer                 18
Name: count, dtype: int64


In [26]:
cv_final['role_label_final'].value_counts()

role_label_final
python_developer          1292
java_developer            1194
web_developer              958
database_administrator     917
security_analyst           865
systems_administrator      764
project_manager            719
frontend_developer         547
network_administrator      452
software_developer         414
fullstack_developer         30
data_scientist              29
mobile_app_developer        23
cloud_engineer              23
backend_developer           19
ml_engineer                 18
Name: count, dtype: int64